In [1]:
## !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

import nltk
import spacy

#from tqdm.auto import tqdm 
#tqdm.pandas()

In [3]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from spacytextblob.spacytextblob import SpacyTextBlob

@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)


In [4]:
nlp = spacy.load("en_core_web_sm")

nlp.add_pipe('sentencizer')
nlp.add_pipe('language_detector')
nlp.add_pipe('spacytextblob')

In [9]:
df = pd.read_csv(r"https://www.ecb.europa.eu/press/key/shared/data/all_ECB_speeches.csv?848ea64ce6d77827b5e8e18790878b64",index_col = ["date"] , sep = "|",encoding='utf-8')

KeyboardInterrupt: 

In [ ]:
df = df.dropna()
df = df[df["contents"]!=" "]
df.sort_values("contents")

In [ ]:
df["nlp"] = df["contents"].apply(lambda x: nlp(x))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\alexj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-9d38fb44b707>", line 1, in <module>
    df["nlp"] = df["contents"].apply(lambda x: nlp(x))
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\apply.py", line 1043, in apply
    return self.apply_standard()
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\apply.py", line 1098, in apply_standard
    mapped = lib.map_infer(
  File "pandas\_libs\lib.pyx", line 2859, in pandas._libs.lib.map_infer
  File "<ipython-input-7-9d38fb44b707>", line 1, in <lambda>
    df["nlp"] = df["contents"].apply(lambda x: nlp(x))
  File "C:\Users\alexj\anaconda3\lib\site-packages\spacy\language.py", li

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\alexj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-9d38fb44b707>", line 1, in <module>
    df["nlp"] = df["contents"].apply(lambda x: nlp(x))
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\apply.py", line 1043, in apply
    return self.apply_standard()
  File "C:\Users\alexj\anaconda3\lib\site-packages\pandas\core\apply.py", line 1098, in apply_standard
    mapped = lib.map_infer(
  File "pandas\_libs\lib.pyx", line 2859, in pandas._libs.lib.map_infer
  File "<ipython-input-7-9d38fb44b707>", line 1, in <lambda>
    df["nlp"] = df["contents"].apply(lambda x: nlp(x))
  File "C:\Users\alexj\anaconda3\lib\site-packages\spacy\language.py", li

In [7]:
df= df[df['nlp'].apply(lambda x: x._.language['language']=="en")]

ERROR! Session/line number was not unique in database. History logging moved to new session 32


KeyError: 'nlp'

In [ ]:
text = [str(x) for x in list(df["nlp"].sample(1)[0].sents)]

<bound method DataFrame.sort_values of                         speakers  \
date                               
2021-12-10         Fabio Panetta   
2021-12-08       Isabel Schnabel   
2021-12-08       Luis de Guindos   
2021-12-08     Christine Lagarde   
2021-11-29     Christine Lagarde   
...                          ...   
1997-05-13  Alexandre Lamfalussy   
1997-04-30  Alexandre Lamfalussy   
1997-04-22  Alexandre Lamfalussy   
1997-03-10  Alexandre Lamfalussy   
1997-02-07  Alexandre Lamfalussy   

                                                        title  \
date                                                            
2021-12-10  The present and future of money in the digital...   
2021-12-08            Monetary policy and financial stability   
2021-12-08  Macroprudential policy for non-bank financial ...   
2021-12-08  Macroprudential policy in Europe – the future ...   
2021-11-29                       Dialogue in a changing world   
...                                  

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

sentences = ["I represent the BCE", 
             "the sky is grey", 
             "the sky is blue", 
             "I am happy"]

inputs = tokenizer(text, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]

labels = {0:'neutral', 1:'positive',2:'negative'}
for idx, sent in enumerate(text):
    print(sent, '----', outputs.detach().numpy()[idx])

In [ ]:
score = {0:0, 1:1, 2:-1}
np.vectorize(lambda x: score[x])(np.argmax(outputs.detach().numpy(), axis = 1))

In [ ]:
[nlp(x)._.polarity for x in text]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words="english")
tfidf_vector = v.fit_transform(df["nlp"].apply(lambda x: x.text))


In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("happiness")

In [ ]:
def clean_docs(texts, remove_stopwords=True, n_process = 4):
    nlp = spacy.load("en_core_web_sm")
    docs = nlp.pipe(texts, 
                    disable=['parser','lemmatizer' ,'ner','textcat'])
    stopwords = nlp.Defaults.stop_words

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [ps.stem(tok) for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

In [ ]:
aaa = df["contents"].sample(1)
cleaned = clean_docs(aaa, remove_stopwords=True)
cleaned

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


In [ ]:
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))

In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()


In [ ]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})


In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
 
# choose some words to be stemmed
words = ["I", "am","happy","but","he","is","not"]
for word in words:
    print(ps.stem(lemmatizer.lemmatize(word)))

In [ ]:
tfidf_df = tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)
tfidf_df

In [ ]:
ps.stem("Hungarian")